# K-crownether analysis

### A demonstration of how the region centers from a WESTPA-WExplore run can be extracted and analyzed

The progress coordinate here is the coordinates of the K+ atom (e.g. the X, Y, Z positions), after alignment to the crownether.  WExplore used a three-level hierarchy of Voronoi polyhedra, which are each defined by an "image". Here we save the coordinates of all the images in the variables `big_regions`, `med_regions` and `sml_regions`.  We measure distances between the different sets of images, and we end the demonstration by writing trajectories containing these coordinates to "dcd" files, which can be visualized in VMD or pymol.

In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

import os
import sys

WEST_ROOT = os.environ['WEST_ROOT']
for lib in ['lib/wwmgr', 'src', 'lib/west_tools']:
    path = os.path.join(WEST_ROOT, lib)
    if path not in sys.path:
        sys.path.append(path)

import west
        
%matplotlib inline

In [ ]:
# the data manager helps access simulation data written to the "west.h5" file
data_manager = west.rc.get_data_manager()
data_manager.open_backing(mode='r')

In [ ]:
string_hashes = data_manager.we_h5file['bin_topologies']['index'][:]['hash']
all_mappers = []
for si, shash in enumerate(string_hashes):
    bin_mapper = data_manager.get_bin_mapper(shash)
    all_mappers.append(bin_mapper)

In [ ]:
latest_mapper = all_mappers[-1]
level_inds = latest_mapper.level_indices

In [ ]:
# get region indices
biggest_regions_inds = level_inds[0]
medium_regions_inds = level_inds[1]
small_regions_inds = level_inds[2]

# find region centers
big_regions = latest_mapper.fetch_centers(biggest_regions_inds)
med_regions = latest_mapper.fetch_centers(medium_regions_inds)
sml_regions = latest_mapper.fetch_centers(small_regions_inds)

print("There are {0} big regions, {1} medium regions, and {2} large regions".format(len(big_regions),len(med_regions),len(sml_regions)))

In [ ]:
# let's calculate the distance between all of the medium regions

n_med = len(med_regions)
dists = np.zeros((n_med,n_med))
for i, x in enumerate(med_regions):
    for j, y in enumerate(med_regions):
        dists[i,j] = np.linalg.norm((x-y))

In [ ]:
dists

In [ ]:
# great!  now let's write these to trajectories
import mdtraj as mdj

ion_pdb = mdj.load_pdb('18-crown-6-K+.pdb')

big_reg_reshape = big_regions.reshape((len(big_regions),ion_pdb.n_atoms,3))/10
med_reg_reshape = med_regions.reshape((len(med_regions),ion_pdb.n_atoms,3))/10
sml_reg_reshape = sml_regions.reshape((len(sml_regions),ion_pdb.n_atoms,3))/10

mdj.Trajectory(big_reg_reshape,ion_pdb.top).save_dcd('big_regions.dcd')
mdj.Trajectory(med_reg_reshape,ion_pdb.top).save_dcd('med_regions.dcd')
mdj.Trajectory(sml_reg_reshape,ion_pdb.top).save_dcd('sml_regions.dcd')

### Great, now go ahead and visualize your regions!  They are pre-aligned.

There is a helpful VMD script to accomplish this:

```
vmd -e vis_align.tcl
```

### After running this script, and doing a little decorating, we get this:

![](all_regions-01.png)